In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from procedures import create_binance_bot, create_binance_bot_spot, make_get_filepath, create_bitget_bot, \
    prepare_backtest_config
from pure_funcs import get_template_live_config, ts_to_date, get_daily_from_income, get_dummy_settings
from njit_funcs import round_dynamic
from time import time, sleep
import os
import pandas as pd
import asyncio
import numpy as np
import matplotlib.pyplot as plt
from downloader import Downloader, load_hlc_cache

In [27]:
plt.rcParams["figure.figsize"] = [21, 13]
pd.set_option("display.precision", 10)

In [30]:
class Args:
    def __init__(self):
        self.backtest_config_path = "configs/backtest/myconfig_v02.hjson"
        self.exchange = "bitget"
        self.symbols = "TRBUSDT"
        self.market_type = "futures"
        self.user = "bitget_01"
        self.start_date = "2023-10-13"
        self.end_date = "2023-10-16"
        self.starting_balance = 1000.0
        self.starting_configs = ""
        self.base_dir = "backtests"
        self.ohlcv = "y"  # set to "n" to use 1s candles


config = prepare_backtest_config(Args())

config["adg_n_subdivisions"] = 20  # see configs/backtest/default.hjson for details
config["inverse"] = False
sts = time()
if config["ohlcv"]:
    data = await load_hlc_cache(
        config["symbol"],
        config["inverse"],
        config["start_date"],
        config["end_date"],
        base_dir=config["base_dir"],
        spot=config["spot"],
        exchange=config["exchange"],
    )
    prices = data[:, 3]
    highs = data[:, 1]
    lows = data[:, 2]
    closes = data[:, 3]
else:
    downloader = Downloader(config)
    data = await downloader.get_sampled_ticks()
    prices = data[:, 2]
    highs = data[:, 2]
    lows = data[:, 2]
    closes = data[:, 2]
timestamps = data[:, 0]
config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f"millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms")

fetching market_specific_settings for TRBUSDT...
TRBUSDT most n days of consecutive identical ohlcvs: 0.000, index last: 0
millis to load 4321 ticks 6ms


In [33]:
type(data)

numpy.ndarray

In [19]:
config = get_template_live_config()
config["symbol"] = "XMRUSDT"  # dummy symbol
config["exchange"] = "bitget"
config["market_type"] = "futures"
config["user"] = "bitget_01"
config["ohlcv"] = True
tmp_live_settings = get_dummy_settings(config)

In [16]:
config

{'config_name': 'neat_template',
 'logging_level': 0,
 'long': {'auto_unstuck_delay_minutes': 300.0,
  'auto_unstuck_ema_dist': 0.02,
  'auto_unstuck_qty_pct': 0.04,
  'auto_unstuck_wallet_exposure_threshold': 0.1,
  'backwards_tp': False,
  'ema_span_0': 1440,
  'ema_span_1': 4320,
  'enabled': True,
  'eprice_exp_base': 1.618034,
  'eqty_exp_base': 1.8,
  'grid_span': 0.16,
  'initial_eprice_ema_dist': -0.01,
  'initial_qty_pct': 0.01,
  'markup_range': 0.0075,
  'max_n_entry_orders': 10,
  'min_markup': 0.0045,
  'n_close_orders': 7,
  'wallet_exposure_limit': 1.6},
 'short': {'auto_unstuck_delay_minutes': 300.0,
  'auto_unstuck_ema_dist': 0.02,
  'auto_unstuck_qty_pct': 0.04,
  'auto_unstuck_wallet_exposure_threshold': 0.1,
  'backwards_tp': False,
  'ema_span_0': 1440,
  'ema_span_1': 4320,
  'enabled': True,
  'eprice_exp_base': 1.618034,
  'eqty_exp_base': 1.8,
  'grid_span': 0.16,
  'initial_eprice_ema_dist': -0.01,
  'initial_qty_pct': 0.01,
  'markup_range': 0.0075,
  'max_n_

In [18]:
config["ohlcv"]

KeyError: 'ohlcv'

In [25]:

try:
    await bot.session.close()
except:
    pass
if config["market_type"] == "spot":
    bot = await create_binance_bot_spot(config)
else:
    bot = await create_binance_bot(tmp_live_settings)
    
    # bot = await create_bitget_bot(tmp_live_settings)
    # settings_from_exchange["maker_fee"] = 0.0002
    # settings_from_exchange["taker_fee"] = 0.0006
    # settings_from_exchange["exchange"] = "bitget"
    

linear perpetual
initiating position, open orders, fills, exchange config, order book, and emas...


ERROR:root:error fetching open orders string indices must be integers
Traceback (most recent call last):
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 304, in fetch_open_orders
    return [
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 306, in <listcomp>
    "order_id": int(e["orderId"]),
TypeError: string indices must be integers
ERROR:root:error with update open orders 'bool' object is not iterable
Traceback (most recent call last):
  File "/home/tdb/git/passivbot/passivbot.py", line 360, in update_open_orders
    open_orders = [x for x in open_orders if x["symbol"] == self.symbol]
TypeError: 'bool' object is not iterable
Traceback (most recent call last):
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 561, in fetch_fills
    fills = [
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 563, in <listcomp>
    "symbol": x["symbol"],
TypeError: string indices must be integers


returned: {'code': -2014, 'msg': 'API-key format invalid.'}
error fetching fills string indices must be integers


Traceback (most recent call last):
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 506, in fetch_latest_fills
    fills += [
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 508, in <listcomp>
    "order_id": elm["orderId"],
TypeError: string indices must be integers
ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 337, in fetch_position
    assert all(
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 338, in <genexpr>
    key in positions[0] for key in ["symbol", "positionAmt", "entryPrice"]
KeyError: 0
ERROR:root:error with update position 


error fetching latest fills string indices must be integers
returned: {'code': -2014, 'msg': 'API-key format invalid.'}
2024-01-09T00:43:54  {'code': -2014, 'msg': 'API-key format invalid.'} 
returned: {'code': -2014, 'msg': 'API-key format invalid.'}
returned: {'code': -2014, 'msg': 'API-key format invalid.'}


Traceback (most recent call last):
  File "/home/tdb/git/passivbot/passivbot.py", line 428, in update_position
    assert position is not None
AssertionError


2024-01-09T00:43:54  {'code': -2014, 'msg': 'API-key format invalid.'} 
2024-01-09T00:43:55  {'code': -2014, 'msg': 'API-key format invalid.'} 
done


ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 337, in fetch_position
    assert all(
  File "/home/tdb/git/passivbot/exchanges/binance.py", line 338, in <genexpr>
    key in positions[0] for key in ["symbol", "positionAmt", "entryPrice"]
KeyError: 0
ERROR:root:error with update position 


returned: {'code': -2014, 'msg': 'API-key format invalid.'}
returned: {'code': -2014, 'msg': 'API-key format invalid.'}


Traceback (most recent call last):
  File "/home/tdb/git/passivbot/passivbot.py", line 428, in update_position
    assert position is not None
AssertionError


In [24]:
bot.endpoints["exchange_info"]

'/api/mix/v1/market/contracts'

In [32]:
info

{'code': '40019',
 'msg': 'Parameter productType cannot be empty',
 'requestTime': 1704760960032,
 'data': None}

In [22]:
info = await bot.public_get(bot.endpoints["exchange_info"])
csvs = {}
syms = [x["symbol"] for x in info["symbols"] if x["symbol"].endswith("USDT")]
n = 7
interval = "5m"
while True:
    if not syms:
        break
    subset = syms[:n]
    syms = syms[n:]
    res = await asyncio.gather(*[bot.fetch_ohlcvs(symbol=s, interval=interval) for s in subset])
    for s, r in zip(subset, res):
        csvs[s] = pd.DataFrame(r)
    print(subset, len(syms))

KeyError: 'symbols'

In [ ]:
def correlation(x, y):
    x_mean = x.mean()
    y_mean = y.mean()
    a = x - x_mean
    b = y - y_mean
    return ((a * b).sum()) / (((a ** 2).sum() * (b ** 2).sum()) ** 0.5)

In [ ]:
n_symbols = 30
vols = [(csvs[s].volume * csvs[s].close).sum() for s in csvs]
minvol = sorted(vols)[-n_symbols]
ss = [s for s in csvs if (csvs[s].volume * csvs[s].close).sum() >= minvol]
corrs = []
corrs_dict = {}
done = set()
for s0 in sorted(ss):
    for s1 in sorted(ss):
        if s0 == s1:
            continue
        key = tuple(sorted([s0, s1]))
        if key in done:
            continue
        done.add(key)
        x = csvs[s0].close
        y = csvs[s1].close
        corr = correlation(x, y)
        print(f"\r{s0} {s1} {corr:.4f}      ", end=" ")
        corrs.append((corr,) + key)

corrs_dict = {}
corrs_dict_abs = {}
for e in corrs:
    if e[1] not in corrs_dict:
        corrs_dict[e[1]] = {e[2]: e[0]}
        corrs_dict_abs[e[1]] = {e[2]: abs(e[0])}

    else:
        corrs_dict[e[1]][e[2]] = e[0]
        corrs_dict_abs[e[1]][e[2]] = abs(e[0])

    if e[2] not in corrs_dict:
        corrs_dict[e[2]] = {e[1]: e[0]}
        corrs_dict_abs[e[2]] = {e[1]: abs(e[0])}

    else:
        corrs_dict[e[2]][e[1]] = e[0]
        corrs_dict_abs[e[2]][e[1]] = abs(e[0])

In [ ]:
# sum correlation of each symbol to every other symbol
corr_sorted = sorted([(sum(corrs_dict[s].values()), s) for s in corrs_dict])
corr_sorted

In [ ]:
# mean of all symbol
mean_prices = pd.concat([e.close / e.close.iloc[0] for e in csvs.values()], axis=1).mean(axis=1)

# compare individual symbol to mean
symbol = "SXPUSDT"
# symbol = corr_sorted[0][1]  # least correlated symbol
mean_prices.rename("mean_prices").plot()
(csvs[symbol].close / csvs[symbol].close.iloc[0]).rename(symbol).plot()
plt.legend()

In [ ]:
# compare two symbols
symbol0 = "BNBUSDT"
symbol1 = "LINKUSDT"
(csvs[symbol0].close / csvs[symbol0].close.iloc[0]).rename(symbol0).plot()
(csvs[symbol1].close / csvs[symbol1].close.iloc[0]).rename(symbol1).plot()
plt.legend()